# Text classification of movie reviews with Keras and TensorFlow Hub

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.0.0-beta1
Eager mode:  True
Hub version:  0.4.0
GPU is NOT AVAILABLE


In [4]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(name="imdb_reviews", split=(train_validation_split, tfds.Split.TEST), as_supervised=True)

In [5]:
# train_examples_batch, train_labels_batch = train_data.batch(10).make_one_shot_iterator().get_next()
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=220, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [6]:
train_labels_batch

<tf.Tensor: id=221, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=402, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20


W0817 02:08:31.754357  4736 deprecation.py:323] From D:\Software\Anaconda3\envs\py3-TF2.0\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================].5133 - accuracy: 0.46 - 3s 1s/step - loss: 3.4048 - accuracy: 0.46 - 3s 1s/step - loss: 3.2036 - accuracy: 0.48 - 3s 855ms/step - loss: 3.0575 - accuracy: 0.495 - 4s 724ms/step - loss: 2.9605 - accuracy: 0.496 - 4s 642ms/step - loss: 2.8480 - accuracy: 0.501 - 4s 575ms/step - loss: 2.7872 - accuracy: 0.499 - 4s 528ms/step - loss: 2.7157 - accuracy: 0.494 - 4s 493ms/step - loss: 2.6460 - accuracy: 0.490 - 5s 459ms/step - loss: 2.5704 - accuracy: 0.490 - 5s 431ms/step - loss: 2.4877 - accuracy: 0.493 - 5s 407ms/step - loss: 2.4278 - accuracy: 0.490 - 5s 386ms/step - loss: 2.3527 - accuracy: 0.491 - 5s 370ms/step - loss: 2.2793 - accuracy: 0.491 - 5s 356ms/step - loss: 2.2127 - accuracy: 0.490 - 5s 343ms/step - loss: 2.1447 - accuracy: 0.491 - 6s 332ms/step - loss: 2.0769 - accuracy: 0.493 - 6s 322ms/step - loss: 2.0151 - accuracy: 0.493 - 6s 313ms/step - loss: 1.9588 - accuracy: 0.491 - 6s 304ms/step - loss: 1.9041 - accuracy: 0.492 - 6s 296ms/step 

30/30 [==============================] - ETA: 41s - loss: 0.5232 - accuracy: 0.748 - ETA: 23s - loss: 0.5311 - accuracy: 0.733 - ETA: 17s - loss: 0.5353 - accuracy: 0.736 - ETA: 13s - loss: 0.5363 - accuracy: 0.732 - ETA: 11s - loss: 0.5363 - accuracy: 0.732 - ETA: 9s - loss: 0.5349 - accuracy: 0.734 - ETA: 8s - loss: 0.5363 - accuracy: 0.73 - ETA: 7s - loss: 0.5355 - accuracy: 0.73 - ETA: 7s - loss: 0.5354 - accuracy: 0.73 - ETA: 6s - loss: 0.5318 - accuracy: 0.74 - ETA: 5s - loss: 0.5312 - accuracy: 0.74 - ETA: 5s - loss: 0.5325 - accuracy: 0.74 - ETA: 4s - loss: 0.5324 - accuracy: 0.74 - ETA: 4s - loss: 0.5302 - accuracy: 0.74 - ETA: 3s - loss: 0.5291 - accuracy: 0.74 - ETA: 3s - loss: 0.5283 - accuracy: 0.74 - ETA: 3s - loss: 0.5275 - accuracy: 0.74 - ETA: 2s - loss: 0.5259 - accuracy: 0.74 - ETA: 2s - loss: 0.5248 - accuracy: 0.74 - ETA: 2s - loss: 0.5241 - accuracy: 0.74 - ETA: 2s - loss: 0.5241 - accuracy: 0.74 - ETA: 1s - loss: 0.5233 - accuracy: 0.75 - ETA: 1s - loss: 0.5238 -

Epoch 12/20
30/30 [==============================] - ETA: 43s - loss: 0.3655 - accuracy: 0.847 - ETA: 24s - loss: 0.3682 - accuracy: 0.849 - ETA: 17s - loss: 0.3707 - accuracy: 0.847 - ETA: 14s - loss: 0.3673 - accuracy: 0.847 - ETA: 11s - loss: 0.3701 - accuracy: 0.846 - ETA: 10s - loss: 0.3749 - accuracy: 0.844 - ETA: 9s - loss: 0.3739 - accuracy: 0.847 - ETA: 8s - loss: 0.3718 - accuracy: 0.84 - ETA: 7s - loss: 0.3727 - accuracy: 0.84 - ETA: 6s - loss: 0.3688 - accuracy: 0.85 - ETA: 6s - loss: 0.3675 - accuracy: 0.85 - ETA: 5s - loss: 0.3662 - accuracy: 0.85 - ETA: 4s - loss: 0.3673 - accuracy: 0.85 - ETA: 4s - loss: 0.3645 - accuracy: 0.85 - ETA: 4s - loss: 0.3635 - accuracy: 0.85 - ETA: 3s - loss: 0.3637 - accuracy: 0.85 - ETA: 3s - loss: 0.3635 - accuracy: 0.85 - ETA: 3s - loss: 0.3620 - accuracy: 0.85 - ETA: 2s - loss: 0.3610 - accuracy: 0.85 - ETA: 2s - loss: 0.3597 - accuracy: 0.86 - ETA: 2s - loss: 0.3595 - accuracy: 0.85 - ETA: 1s - loss: 0.3588 - accuracy: 0.86 - ETA: 1s - 

Epoch 18/20
30/30 [==============================] - ETA: 41s - loss: 0.2519 - accuracy: 0.898 - ETA: 23s - loss: 0.2497 - accuracy: 0.901 - ETA: 16s - loss: 0.2485 - accuracy: 0.904 - ETA: 13s - loss: 0.2438 - accuracy: 0.906 - ETA: 11s - loss: 0.2484 - accuracy: 0.903 - ETA: 10s - loss: 0.2555 - accuracy: 0.900 - ETA: 8s - loss: 0.2553 - accuracy: 0.901 - ETA: 8s - loss: 0.2530 - accuracy: 0.90 - ETA: 7s - loss: 0.2542 - accuracy: 0.90 - ETA: 6s - loss: 0.2510 - accuracy: 0.90 - ETA: 5s - loss: 0.2494 - accuracy: 0.90 - ETA: 5s - loss: 0.2474 - accuracy: 0.90 - ETA: 4s - loss: 0.2489 - accuracy: 0.90 - ETA: 4s - loss: 0.2466 - accuracy: 0.91 - ETA: 4s - loss: 0.2461 - accuracy: 0.91 - ETA: 3s - loss: 0.2464 - accuracy: 0.91 - ETA: 3s - loss: 0.2464 - accuracy: 0.90 - ETA: 2s - loss: 0.2457 - accuracy: 0.91 - ETA: 2s - loss: 0.2446 - accuracy: 0.91 - ETA: 2s - loss: 0.2434 - accuracy: 0.91 - ETA: 2s - loss: 0.2433 - accuracy: 0.91 - ETA: 1s - loss: 0.2428 - accuracy: 0.91 - ETA: 1s - 

In [11]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.326
accuracy: 0.860
